# Download Shapefile

In [ ]:
%%html 
<style>
.tooltip {
    position: relative;
    display: inline-block;
    border-bottom: 1px dotted black;
}

.tooltip .tooltiptext {
    visibility: hidden;
    width: 220px;
    background-color: white;
    color: black;
    text-align: center;
    border-radius: 6px;
    padding: 5px 0;
    border: 1px solid black;
    /* Position the tooltip */
    position: absolute;
    z-index: 1;
}

.tooltip:hover .tooltiptext {
    visibility: visible;
}
</style>
<div class="tooltip">
  <span class="tooltiptext">
<img src="https://upload.wikimedia.org/wikipedia/commons/3/38/Simple_vector_map.svg" style="max-width:100%;">
</span>
</div>


In [ ]:
from IPython.display import  HTML

def load_d3_in_cell_output():
  display(HTML("<script src='https://d3js.org/d3.v6.min.js'></script>"))
get_ipython().events.register('pre_run_cell', load_d3_in_cell_output)

In [ ]:
%%html
<script type="text/javascript">   

    d3.json("https://gist.githubusercontent.com/dudaspm/89da9e990236d2bc73a3a6ee00c18bb6/raw/055b78a9d016873c36e28872ab6a4d85d0be858e/usCounties.json")
      .then(function(map) { console.log(map) });
   
</script>

Where to get the csv of our data?? 

In [ ]:
%%html
<div id="gohere4"></div>
<script type="text/javascript">   

    d3.json("https://gist.githubusercontent.com/dudaspm/89da9e990236d2bc73a3a6ee00c18bb6/raw/055b78a9d016873c36e28872ab6a4d85d0be858e/usCounties.json")
      .then(function(map) { 
        d3.csv("https://gist.githubusercontent.com/dudaspm/57951a2fa648fa23b4197768bbac2907/raw/ef63f7bf27d7dbcfffafe5deebe3b28f2ce5ddf0/data_EPA_county.csv")
          .then(function(data) { 
            const chooseState = 36
            const epa = data.map(d=> ({fips:d.fips,pct_pub_transport:+d.pct_pub_transport}) )
            const PAEPAData = epa.filter(d=> d.fips.substring(0, 2) == chooseState) 
            const color = d3.scaleLinear().range([0,1]).domain(d3.extent(PAEPAData,d=>+d.pct_pub_transport));
            const palette = d3.interpolateOranges; 
            const width = 600
            const height = 400
            const margin = 0
            
            const ny = ({type:"FeatureCollection", features:map.features.filter(d=> d.properties.STATEFP==chooseState)})

            const projectionNY = d3.geoMercator().fitExtent([[margin, margin], [width - margin, height - margin]], ny)
            const pathGeneratorNY = d3.geoPath().projection(projectionNY)

            const svg = d3.select("div#gohere4").append("svg")
                .attr("width", width)
                .attr("height", height)
            var EPAKeys = {}
            PAEPAData.forEach((d,i)=> EPAKeys[d.fips] = d.pct_pub_transport) 
            console.log(EPAKeys)
            // construct the element
            svg.selectAll("path").append('path')
              .data(ny.features)
              .join("path")
              .attr('d', pathGeneratorNY)
              .attr("fill", d=>palette(color(EPAKeys[d.properties.GEOID])))
              .attr('stroke', '#999999')
              .attr('stroke-width', '2')
                .append("title")
                .text(d => "the percent public transportation is: \n" + EPAKeys[d.properties.GEOID]);


          })
       });
   
</script>

### Adding Zoom Feature

In [ ]:
%%html
<div id="gohere5"></div>
<script type="text/javascript">   

    d3.json("https://gist.githubusercontent.com/dudaspm/89da9e990236d2bc73a3a6ee00c18bb6/raw/055b78a9d016873c36e28872ab6a4d85d0be858e/usCounties.json")
      .then(function(map) { 
        d3.csv("https://raw.githubusercontent.com/ArwaHarar/DS440/main/coords_density_df.csv")
          .then(function(data) { 
            console.log(data)
            const chooseState = 36
            const tweets = data.map(d=> ({Coords:d.Coords.replace("[","").replace("]","").split(","),Timestamp:d.Timestamp,Tweet:d.Tweet,Density:+d.Density}) )
            console.log(tweets)
            //const PAEPAData = tweets.filter(d=> d.fips.substring(0, 2) == chooseState) 
            //const color = d3.scaleLinear().range([0,1]).domain(d3.extent(PAEPAData,d=>+d.pct_pub_transport));
            const palette = d3.interpolateOranges; 
            const width = 600
            const height = 400
            const margin = 0
            
            const ny = ({type:"FeatureCollection", features:map.features.filter(d=> d.properties.STATEFP==chooseState)})

            const projectionNY = d3.geoMercator().fitExtent([[margin, margin], [width - margin, height - margin]], ny)
            const pathGeneratorNY = d3.geoPath().projection(projectionNY)

            const svg = d3.select("div#gohere5").append("svg")
                .attr("width", width)
                .attr("height", height)
            var EPAKeys = {}
            //PAEPAData.forEach((d,i)=> EPAKeys[d.fips] = d.pct_pub_transport) 



            ////////////////////////////////////////
            // ZOOOOOOOOOOOOOOOOOOOOOOOOMING
            svg.call(d3.zoom()
                .extent([[0, 0], [width, height]])
                .scaleExtent([1, 15])
                .on("zoom", zoomed));

            function zoomed({transform}) {
              g.attr("transform", transform);
            }
            const g = svg.append("g");
            ////////////////////////////////////////
            // construct the element
            g.selectAll("path").append('path')
              .data(ny.features)
              .join("path")
              .attr('d', pathGeneratorNY)
              //.attr("fill", d=>palette(color(EPAKeys[d.properties.GEOID])))
              .style("fill","white")
              .attr('stroke', '#999999')
              .attr('stroke-width', '2')

            g.selectAll("circle").append('circle')
              .data(tweets)
              .join("circle")
              //.attr("cx",d=> console.log(projectionNY([d.Coords[0],d.Coords[1]])[0])))
              .attr("cx",d=> projectionNY([d.Coords[0],d.Coords[1]])[0])
              .attr("cy",d=> projectionNY([d.Coords[0],d.Coords[1]])[1])
              .attr("r",1)
              .style("fill","Blue")
              .style("opacity",.05)
                .append("title")
                .text(d => d.Density);
              

          })
       });
   
</script>

In [ ]:
%%html
<div id="gohere6"></div>
<script type="module"> 
    import * as d3 from "https://cdn.skypack.dev/d3@7"; 
    import {tile} from "https://cdn.skypack.dev/d3-tile@1";

    d3.json("https://gist.githubusercontent.com/dudaspm/89da9e990236d2bc73a3a6ee00c18bb6/raw/055b78a9d016873c36e28872ab6a4d85d0be858e/usCounties.json")
      .then(function(map) { 
        d3.csv("https://gist.githubusercontent.com/dudaspm/57951a2fa648fa23b4197768bbac2907/raw/ef63f7bf27d7dbcfffafe5deebe3b28f2ce5ddf0/data_EPA_county.csv")
          .then(function(data) { 
            const chooseState = 36
            const epa = data.map(d=> ({fips:d.fips,pct_pub_transport:+d.pct_pub_transport}) )
            const PAEPAData = epa.filter(d=> d.fips.substring(0, 2) == chooseState) 
            const color = d3.scaleLinear().range([0,1]).domain(d3.extent(PAEPAData,d=>+d.pct_pub_transport));
            const palette = d3.interpolateOranges; 
            const width = 600
            const height = 400
            const margin = 0
            
            const ny = ({type:"FeatureCollection", features:map.features.filter(d=> d.properties.STATEFP==chooseState)})

            const projectionNY = d3.geoMercator().fitExtent([[margin, margin], [width - margin, height - margin]], ny)
            const pathGeneratorNY = d3.geoPath().projection(projectionNY)
            
            const svg = d3.select("div#gohere6").append("svg")
                .attr("width", width)
                .attr("height", height)

            var EPAKeys = {}
            PAEPAData.forEach((d,i)=> EPAKeys[d.fips] = d.pct_pub_transport) 
            console.log(EPAKeys)
            // construct the element

            const tiles = tile()
                .extent([[0, 0], [width, height]])
                .tileSize(512)
                .clampX(false);
            const zoom = d3.zoom()
                .scaleExtent([1 << 12, 1 << 20])
                .extent([[0, 0], [width, height]])
                .on("zoom", ({transform}) => zoomed(transform));


            console.log("44") 
            let image = svg.append("g")
                .attr("pointer-events", "none")
              .selectAll("image");
            console.log("48")

    
            console.log("55")

            const path = svg.append("path")
                .attr("pointer-events", "none")
                .attr("fill", "none")
                .attr("stroke", "red")
                .attr("stroke-linecap", "round")
                .attr("stroke-linejoin", "round");

            const initialCenter = [-73.935242,40.730610]
            const initialScale = 1 << 12
            console.log((projectionNY(initialCenter)))
            svg
                .call(zoom)
                .call(zoom.transform, d3.zoomIdentity
                    .translate(width / 2, height / 2)
                    .scale(-initialScale)
                    .translate(...projectionNY(initialCenter))
                    .scale(-1));


            function zoomed(transform) {
              /*const tiles = tile(transform);
              console.log(tiles())

              image = image.data(tiles, d => d).join("image")
                  .attr("xlink:href", d => url(...d3.tileWrap(d)))
                  .attr("x", ([x]) => (x + tiles.translate[0]) * tiles.scale)
                  .attr("y", ([, y]) => (y + tiles.translate[1]) * tiles.scale)
                  .attr("width", tiles.scale)
                  .attr("height", tiles.scale);*/

              console.log("asdf")
              projectionNY
                  .scale(transform.k / (2 * Math.PI))
                  .translate([transform.x, transform.y]);

              path.attr("d", pathGeneratorNY(ny));
            }
            function url(x,y,z) {
              return "http://" + ["a", "b", "c"][Math.random() * 3 | 0] + ".tile.openstreetmap.org/" + z + "/" + x + "/" + y + ".png";
            }

          })
       });
   
</script>

In [ ]:
%%html
<div id="gohere5"></div>
<script type="module"> 
    import * as d3 from "https://cdn.skypack.dev/d3@7"; 
    
    d3.json("https://gist.githubusercontent.com/dudaspm/89da9e990236d2bc73a3a6ee00c18bb6/raw/055b78a9d016873c36e28872ab6a4d85d0be858e/usCounties.json")
      .then(function(map) { 
        d3.csv("https://raw.githubusercontent.com/ArwaHarar/DS440/main/coords_density_df.csv")
          .then(function(data) { 
            console.log(data)
            const chooseState = 36
            const tweets = data.map(d=> ({Coords:d.Coords.replace("[","").replace("]","").split(","),Timestamp:d.Timestamp,Tweet:d.Tweet,Density:+d.Density}) )
            console.log(tweets)
            //const PAEPAData = tweets.filter(d=> d.fips.substring(0, 2) == chooseState) 
            //const color = d3.scaleLinear().range([0,1]).domain(d3.extent(PAEPAData,d=>+d.pct_pub_transport));
            const palette = d3.interpolateOranges; 
            const width = 600
            const height = 400
            const margin = 0
            
            const ny = ({type:"FeatureCollection", features:map.features.filter(d=> d.properties.STATEFP==chooseState)})

            const projectionNY = d3.geoMercator().fitExtent([[margin, margin], [width - margin, height - margin]], ny)
            const pathGeneratorNY = d3.geoPath().projection(projectionNY)

            const svg = d3.select("div#gohere5").append("svg")
                .attr("width", width)
                .attr("height", height)
            var EPAKeys = {}
            //PAEPAData.forEach((d,i)=> EPAKeys[d.fips] = d.pct_pub_transport) 



            ////////////////////////////////////////
            // ZOOOOOOOOOOOOOOOOOOOOOOOOMING
            svg.call(d3.zoom()
                .extent([[0, 0], [width, height]])
                .scaleExtent([1, 15])
                .on("zoom", zoomed));

            function zoomed({transform}) {
              g.attr("transform", transform);
            }
            const g = svg.append("g");
            ////////////////////////////////////////
            // construct the element
            g.selectAll("path").append('path')
              .data(ny.features)
              .join("path")
              .attr('d', pathGeneratorNY)
              //.attr("fill", d=>palette(color(EPAKeys[d.properties.GEOID])))
              .style("fill","white")
              .attr('stroke', '#999999')
              .attr('stroke-width', '2')

            g.selectAll("circle").append('circle')
              .data(tweets)
              .join("circle")
              //.attr("cx",d=> console.log(projectionNY([d.Coords[0],d.Coords[1]])[0])))
              .attr("cx",d=> projectionNY([d.Coords[0],d.Coords[1]])[0])
              .attr("cy",d=> projectionNY([d.Coords[0],d.Coords[1]])[1])
              .attr("r",1)
              .style("fill","Blue")
              .style("opacity",.05)
                .append("title")
                .text(d => d.Density);
              

          })
       });
   
</script>

### From: https://jqueryui.com/slider/#range

In [ ]:
%%html
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Slider - Range slider</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.13.1/themes/base/jquery-ui.css">
  <link rel="stylesheet" href="/resources/demos/style.css">
  <script src="https://code.jquery.com/jquery-3.6.0.js"></script>
  <script src="https://cdn.skypack.dev/d3@7"></script>
  <script src="https://code.jquery.com/ui/1.13.1/jquery-ui.js"></script>

</head>
<body>
 
<p>
  <label for="date">Date range:</label>
  <input type="text" id="date" readonly style="border:0; color:#f6931f; font-weight:bold;">
</p>

<div id="slider-range" style="width:400px"></div>
<div id="output"></div>
<script type="module"> 
    import * as d3 from "https://cdn.skypack.dev/d3@7";

    d3.json("https://gist.githubusercontent.com/dudaspm/89da9e990236d2bc73a3a6ee00c18bb6/raw/055b78a9d016873c36e28872ab6a4d85d0be858e/usCounties.json")
      .then(function(map) { 
        d3.csv("https://raw.githubusercontent.com/ArwaHarar/DS440/main/coords_density_df.csv")
          .then(function(data) { 
            console.log(data)
            const parseTime = d3.timeParse("%B %d, %Y");
            const chooseState = 36
            const tweets = data.map(d=> ({Coords:d.Coords.replace("[","").replace("]","").split(","),Timestamp:Date.parse(d.Timestamp),Tweet:d.Tweet,Density:+d.Density}) )
            console.log(tweets)
            //const PAEPAData = tweets.filter(d=> d.fips.substring(0, 2) == chooseState) 
            //const color = d3.scaleLinear().range([0,1]).domain(d3.extent(PAEPAData,d=>+d.pct_pub_transport));
            const palette = d3.interpolateOranges; 
            const width = 600
            const height = 400
            const margin = 0
            
            const ny = ({type:"FeatureCollection", features:map.features.filter(d=> d.properties.STATEFP==chooseState)})

            const projectionNY = d3.geoMercator().fitExtent([[margin, margin], [width - margin, height - margin]], ny)
            const pathGeneratorNY = d3.geoPath().projection(projectionNY)

            const svg = d3.select("div#output").append("svg")
                .attr("width", width)
                .attr("height", height)
            var EPAKeys = {}
            //PAEPAData.forEach((d,i)=> EPAKeys[d.fips] = d.pct_pub_transport) 



            ////////////////////////////////////////
            // ZOOOOOOOOOOOOOOOOOOOOOOOOMING

            svg
              .call(d3.zoom()
                  .extent([[0, 0], [width, height]])
                  .scaleExtent([1, 15])
                  .on("zoom", zoomed))

            


            function zoomed({transform}) {
              g.attr("transform", transform);
            }
            const g = svg.append("g")
            ////////////////////////////////////////
            // construct the element
            g.selectAll("path").append('path')
              .data(ny.features)
              .join("path")
              .attr('d', pathGeneratorNY)
              //.attr("fill", d=>palette(color(EPAKeys[d.properties.GEOID])))
              .style("fill","white")
              .attr('stroke', '#999999')
              .attr('stroke-width', '2')

            g.selectAll("circle").append('circle')
              .data(tweets)
              .join("circle")
              //.attr("cx",d=> console.log(projectionNY([d.Coords[0],d.Coords[1]])[0])))
              .attr("cx",d=> projectionNY([d.Coords[0],d.Coords[1]])[0])
              .attr("cy",d=> projectionNY([d.Coords[0],d.Coords[1]])[1])
              .attr("r",.5)
              .style("fill","Blue")
              .style("opacity",.05)
                .append("title")
                .text(d => d.Density);
    var maxDate = d3.max(tweets,d=>d.Timestamp)
    var minDate = d3.min(tweets,d=>d.Timestamp)
    console.log(maxDate)
    $( function() {
      $( "#slider-range" ).slider({
          range: true,
          min: minDate,
          max: maxDate,
          values: [ minDate, maxDate ],
          slide: function( event, ui ) {
            var date1 = new Date(ui.values[ 0 ])
            var date2 = new Date(ui.values[ 1 ])
            var fullDate1 = date1.getMonth() + "-" + date1.getUTCDate() + " " + date1.getHours() + ":" + String(date1.getMinutes()).padStart(2, "0") + ":" + date1.getSeconds()
            var fullDate2 = date2.getMonth() + "-" + date2.getUTCDate() + " " + date2.getHours() + ":" + String(date2.getMinutes()).padStart(2, "0") + ":" + date2.getSeconds()
            $( "#date" ).val( fullDate1 + " - " + fullDate2 );

            g.selectAll("circle")
              .style("opacity",d=> ((ui.values[ 0 ]<d.Timestamp)&&(ui.values[ 1 ]>d.Timestamp)) ? .05 : .0)
          }
          });
          var date1 = new Date($( "#slider-range" ).slider( "values", 0 ))
          var date2 = new Date($( "#slider-range" ).slider( "values", 1 ))
          var fullDate1 = date1.getMonth() + "-" + date1.getUTCDate() + " " + date1.getHours() + ":" + String(date1.getMinutes()).padStart(2, "0") + ":" + date1.getSeconds()
          var fullDate2 = date2.getMonth() + "-" + date2.getUTCDate() + " " + date2.getHours() + ":" + String(date2.getMinutes()).padStart(2, "0") + ":" + date2.getSeconds()
          $( "#date" ).val( fullDate1 + " - " + fullDate2 );

        });
          })
       })
  </script>
</body>
</html>

In [ ]:
%%html
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Slider - Range slider</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.13.1/themes/base/jquery-ui.css">
  <link rel="stylesheet" href="/resources/demos/style.css">
  <script src="https://code.jquery.com/jquery-3.6.0.js"></script>
  <script src="https://cdn.skypack.dev/d3@7"></script>
  <script src="https://code.jquery.com/ui/1.13.1/jquery-ui.js"></script>

</head>
<body>
 
<p>
  <label for="date">Date range:</label>
  <input type="text" id="date" readonly style="border:0; color:#f6931f; font-weight:bold;">
</p>

<div id="slider-range" style="width:400px"></div>
<div id="weight"><p>Current Density in Circle: 0</p></div>
<div id="output2"></div>
<script type="module"> 
    import * as d3 from "https://cdn.skypack.dev/d3@7";

    d3.json("https://gist.githubusercontent.com/dudaspm/89da9e990236d2bc73a3a6ee00c18bb6/raw/055b78a9d016873c36e28872ab6a4d85d0be858e/usCounties.json")
      .then(function(map) { 
        d3.csv("https://raw.githubusercontent.com/ArwaHarar/DS440/main/coords_density_df.csv")
          .then(function(data) { 
            const parseTime = d3.timeParse("%B %d, %Y");
            const chooseState = 36
            const tweets = data.map(d=> ({Coords:d.Coords.replace("[","").replace("]","").split(","),Timestamp:Date.parse(d.Timestamp),Tweet:d.Tweet,Density:+d.Density}) )
            //const PAEPAData = tweets.filter(d=> d.fips.substring(0, 2) == chooseState) 
            //const color = d3.scaleLinear().range([0,1]).domain(d3.extent(PAEPAData,d=>+d.pct_pub_transport));
            const palette = d3.interpolateOranges; 
            const width = 600
            const height = 600
            const margin = 0
            
            const ny = ({type:"FeatureCollection", features:map.features.filter(d=> d.properties.STATEFP==chooseState)})

            const projectionNY = d3.geoMercator().fitExtent([[margin, margin], [width - margin, height - margin]], ny)
            const pathGeneratorNY = d3.geoPath().projection(projectionNY)

            const svg = d3.select("div#output2").append("svg")
                .attr("width", width)
                .attr("height", height)
            var EPAKeys = {}
            //PAEPAData.forEach((d,i)=> EPAKeys[d.fips] = d.pct_pub_transport) 



            ////////////////////////////////////////
            // ZOOOOOOOOOOOOOOOOOOOOOOOOMING

            svg
              .call(d3.zoom()
                  .extent([[0, 0], [width, height]])
                  .scaleExtent([1, 15])
                  .on("zoom", zoomed))

            
            var counter = 0
            
            svg.on("mousemove", (event) => {
            var x_p = d3.pointer(event)[0]
            var y_p = d3.pointer(event)[1]
            svg.select("circle.pointer")

              .attr("cx", x_p)
              .attr("cy", y_p)
            if (counter==10) {
                var total = 0
                //console.log(d3.selectAll("circle.markers")._groups[0][0])
                var x_p = d3.pointer(event)[0]
                var y_p = d3.pointer(event)[1]
                svg.select("circle.pointer")

                  .attr("cx", x_p)
                  .attr("cy", y_p)

                var transform = d3.zoomTransform(g.node());  
                var total = 0
                for (let i = 0; i < data.length; i++) {
                  var e = d3.select("circle#index_"+i)
                  var cord = [+e.attr("cx"),+e.attr("cy")]
                  var x_t = (cord[0]*transform.k + transform.x/1)
                  var y_t = (cord[1]*transform.k + transform.y/1)
                  
                  var diff_x = (x_t - x_p)/1
                  var diff_y = (y_t - y_p)/1
                  if (((diff_x*diff_x)+(diff_y*diff_y))<= 100) total = total + (+tweets[i].Density)
                }

                //console.log(((diff_x*diff_x)+(diff_y*diff_y)),transform.k,diff_x,cord[0],transform.x,x,x_p,(x - x_p),"    ",diff_y,cord[1],transform.y,y,y_p,(y - y_p))
                //console.log(((x - x_p) * (x - x_p) + (y - y_p) * (y - y_p)))
                //console.log(x_t,"  ",transform.x,"   ",cord[0])
                document.getElementById("weight").innerHTML = "<p>Current Density in Circle: "+total+"</p>"
                counter = 0
                }
                else {counter = counter + 1}

                
                //if 
              })
            function zoomed({transform}) {
              g.attr("transform", transform);
            }
            const g = svg.append("g")
            ////////////////////////////////////////
            // construct the element
            g.selectAll("path").append('path')
              .data(ny.features)
              .join("path")
              .attr('d', pathGeneratorNY)
              //.attr("fill", d=>palette(color(EPAKeys[d.properties.GEOID])))
              .style("fill","white")
              .attr('stroke', '#999999')
              .attr('stroke-width', '2')

            g.selectAll("circle.markers").append('circle')
              .data(tweets)
              .join("circle")
              .attr("class","markers")
              .attr("id",(d,i)=> "index_"+i)
              //.attr("cx",d=> console.log(projectionNY([d.Coords[0],d.Coords[1]])[0])))
              .attr("cx",d=> projectionNY([d.Coords[0],d.Coords[1]])[0])
              .attr("cy",d=> projectionNY([d.Coords[0],d.Coords[1]])[1])
              .attr("r",.5)
              .attr("data",d=> d.Density)
              .style("fill","Blue")
              .style("opacity",.05)
                .append("title")
                .text(d => d.Density);

              svg.append("circle")
              .attr("class","pointer")
              .attr("r",10)
              .attr("cx", -100)
              .attr("cy", -100)
              .style("fill","none")
              .style("stroke","black")
              .style("stroke-width","1px")

    var maxDate = d3.max(tweets,d=>d.Timestamp)
    var minDate = d3.min(tweets,d=>d.Timestamp)
    console.log(maxDate)
    $( function() {
      $( "#slider-range" ).slider({
          range: true,
          min: minDate,
          max: maxDate,
          values: [ minDate, maxDate ],
          slide: function( event, ui ) {
            var date1 = new Date(ui.values[ 0 ])
            var date2 = new Date(ui.values[ 1 ])
            var fullDate1 = date1.getMonth() + "-" + date1.getUTCDate() + " " + date1.getHours() + ":" + String(date1.getMinutes()).padStart(2, "0") + ":" + date1.getSeconds()
            var fullDate2 = date2.getMonth() + "-" + date2.getUTCDate() + " " + date2.getHours() + ":" + String(date2.getMinutes()).padStart(2, "0") + ":" + date2.getSeconds()
            $( "#date" ).val( fullDate1 + " - " + fullDate2 );

            g.selectAll("circle.markers")
              .style("opacity",d=> ((ui.values[ 0 ]<d.Timestamp)&&(ui.values[ 1 ]>d.Timestamp)) ? .05 : .0)
          }
          });
          var date1 = new Date($( "#slider-range" ).slider( "values", 0 ))
          var date2 = new Date($( "#slider-range" ).slider( "values", 1 ))
          var fullDate1 = date1.getMonth() + "-" + date1.getUTCDate() + " " + date1.getHours() + ":" + String(date1.getMinutes()).padStart(2, "0") + ":" + date1.getSeconds()
          var fullDate2 = date2.getMonth() + "-" + date2.getUTCDate() + " " + date2.getHours() + ":" + String(date2.getMinutes()).padStart(2, "0") + ":" + date2.getSeconds()
          $( "#date" ).val( fullDate1 + " - " + fullDate2 );

        });

function isInside(circle_x, circle_y, rad, x, y)
{
     
    // Compare radius of circle with
    // distance of its center from
    // given point
     
    if ((x - circle_x) * (x - circle_x) +
        (y - circle_y) * (y - circle_y) <= rad * rad)
        return true;
    else
        return false;
}
          })
       })


  </script>
</body>
</html>